### Importaciones y descarga

In [28]:
#INSTALACION DE LIBRERIAS:
#!pip install spacy

#Descaraga en español:
#!pyhton -m spacydownload es_core_news_sm

#Descarga en ingles:
#!python -m spacy download en_core_web_sm

In [29]:
# Bloque 1: Importar librerías y cargar el dataset
import pandas as pd
import en_core_web_sm  # Cambiamos a modelo en inglés
import numpy as np

# Cargar los datos del corpus desde el archivo Excel
df = pd.read_excel('Corpus-Agro.xlsx')

# Eliminar filas con resúmenes vacíos
df = df.dropna(subset=['Resumen'])

# Mostrar las primeras filas del dataset
df.head()


,URL de Documento,Resumen
0,https://faolex.fao.org/docs/pdf/vie78248.pdf,"This Decree provides for the functions, tasks,..."
1,https://faolex.fao.org/docs/pdf/vie95279.pdf,This Decision approves the Scheme on the devel...
2,https://faolex.fao.org/docs/pdf/ita121539.pdf,This Regional Act sets out the legislative fra...
3,https://faolex.fao.org/docs/pdf/cro126979.pdf,This Regulation amends various provisions of t...
4,https://faolex.fao.org/docs/pdf/chn137120C.pdf...,This Law is enacted for the purposes of guaran...


In [31]:
# Bloque 2: Limpiar y lematizar los resúmenes del dataset

# Cargar el modelo de inglés para procesamiento de texto
nlp = en_core_web_sm.load()

# Definir la función para limpiar y lematizar los resúmenes
def limpiar_texto(texto):
    if isinstance(texto, float):  # Comprobar si el texto es un número o NaN
        return ""
    
    # Procesar el texto con spaCy
    doc = nlp(texto.lower())  # Convertir a minúsculas y procesar con spaCy

    # Filtrar stopwords y puntuación, lematizar
    palabras_limpias = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return " ".join(palabras_limpias)

# Aplicar la función de limpieza a todos los resúmenes
df['Resumen_Limpio'] = df['Resumen'].apply(limpiar_texto)

# Mostrar algunos resultados lematizados
df[['Resumen', 'Resumen_Limpio']].head()



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Bloque 3: Entrenar el modelo Word2Vec

from gensim.models import Word2Vec

# Preparar el corpus para el modelo Word2Vec (convertir resúmenes a listas de palabras)
corpus = [resumen.split() for resumen in df['Resumen_Limpio']]

# Entrenar el modelo Word2Vec
modelo_w2v = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

# Guardar el modelo entrenado
modelo_w2v.save("modelo_agro_word2vec.model")


In [ ]:
# Bloque 4: Solicitar una consulta al usuario

# Solicitar una consulta al usuario
consulta = input("Ingresa tu consulta de búsqueda sobre el área de agroindustria: ")

# Limpiar y lematizar la consulta
consulta_limpia = limpiar_texto(consulta)
consulta_tokens = consulta_limpia.split()

# Mostrar el texto limpio de la consulta
print("Consulta lematizada y limpia:", consulta_limpia)


In [ ]:
# Bloque 5: Convertir la consulta a su vector

import numpy as np

# Obtener el vector promedio de las palabras en la consulta
def vectorizar_texto(texto_tokens, modelo):
    vectores = [modelo.wv[token] for token in texto_tokens if token in modelo.wv]
    
    if len(vectores) == 0:  # Si no hay palabras en el vocabulario
        return np.zeros(modelo.vector_size)
    
    return np.mean(vectores, axis=0)

# Vector de la consulta
vector_consulta = vectorizar_texto(consulta_tokens, modelo_w2v)

# Mostrar el vector resultante
print("Vector de la consulta:", vector_consulta)


In [5]:
# Bloque 6: Comparar la consulta con los documentos usando similitud coseno

from sklearn.metrics.pairwise import cosine_similarity

# Vectorizar todos los resúmenes del corpus
df['Vector_Resumen'] = df['Resumen_Limpio'].apply(lambda resumen: vectorizar_texto(resumen.split(), modelo_w2v))

# Calcular la similitud coseno entre la consulta y los resúmenes
df['Similitud'] = df['Vector_Resumen'].apply(lambda vector: cosine_similarity([vector_consulta], [vector])[0][0])

# Mostrar los resúmenes más similares
df_resultados = df[['URL del Documento', 'Resumen', 'Similitud']].sort_values(by='Similitud', ascending=False)
df_resultados.head(10)  # Mostrar los 10 resúmenes más similares


In [6]:
# Bloque 7: Mostrar los mejores textos ordenados por similitud

# Definir cuántos resultados mostrar (n mejores)
n_resultados = 10

# Mostrar los n mejores resultados
df_resultados.head(n_resultados)
